In [253]:
# importing the neceessary packages
import pandas as pd
pd.set_option('display.max_columns', 300)
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy as sp
from scipy.stats import pearsonr, f_oneway
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import metrics, linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
import pickle

## Step 1: Read in hold out data and best model

In [254]:
holdout = pd.read_csv('kc_house_data_test_features.csv', index_col=0)
holdout.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [255]:
final_model = pickle.load(open('model.pickle', 'rb'))

## Step 2: Feature Engineering for holdout set

First, we have to perform all the same transformation on our holdout data that were performed on the original data.

In [256]:
# there is a house with 33 bedrooms, which was confirmed to be false.
# so if any entry has over 20 bedrooms, then replace it with the median.

holdout.loc[holdout.bedrooms > 20, 'bedrooms'] = holdout.bedrooms.median()

In [257]:
## new feature 1: pulling the 'months_old' from 'date' column

# cleaning the dates column, so that it's only the date numbers
holdout["date"] = holdout["date"].str.replace('T000000', '')

# make new column called month_sold with only the months
holdout['month_sold'] = holdout['date'].str[4:6]

# removing 0s only if they are at the begnining of single digit months
holdout['month_sold'] = holdout['month_sold'].str.replace('01', '1').replace('02', '2').replace('03', '3').replace('04', '4').replace('05', '5').replace('06', '6').replace('07', '7').replace('08', '8').replace('09', '9')

# converting new column 'month_sold' to int so we can graph it
holdout['month_sold'] = holdout['month_sold'].astype(int)

In [258]:
## new feature 2: age of the property

# subtracting 'yr_built' with the current year
holdout['age'] = 2020 - holdout['yr_built']

# after i ran this as a graph, there was a crazy outlier so i'll just replace that with the median
holdout.loc[holdout.age < 0] = holdout.age.median()

In [259]:
## new feature 3: does the home have a basement?

# creating dummy variable column 'basement'
holdout['basement'] = holdout['sqft_basement'].apply(lambda x:  1 if x > 0 else 0)

In [260]:
## new feature 4: has the house been renovated?

# creating dummy variable column 'renovated'
holdout['renovated'] = holdout['yr_renovated'].apply(lambda x:  1 if x > 0 else 0)

In [261]:
## new Feature 5: Creating an interaction between `full_sqft` and `waterfront`

holdout['sqft_waterfront'] = holdout['sqft_lot'] * holdout['waterfront']

In [262]:
# checking that everything worked successfully
holdout.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month_sold,age,basement,renovated,sqft_waterfront
0,1.974300e+09,20140827,4.0,2.50,2270.0,11500.0,1.0,0.0,0.0,3.0,8.0,1540.0,730.0,1967.0,0.0,98034.0,47.7089,-122.241,2020.0,10918.0,8.0,53.0,1,0,0.0
1,1.974300e+09,20150218,4.0,2.50,2270.0,11500.0,1.0,0.0,0.0,3.0,8.0,1540.0,730.0,1967.0,0.0,98034.0,47.7089,-122.241,2020.0,10918.0,2.0,53.0,1,0,0.0
2,3.630020e+09,20141107,3.0,2.50,1470.0,1779.0,2.0,0.0,0.0,3.0,8.0,1160.0,310.0,2005.0,0.0,98029.0,47.5472,-121.998,1470.0,1576.0,11.0,15.0,1,0,0.0
3,1.771000e+09,20141203,3.0,1.75,1280.0,16200.0,1.0,0.0,0.0,3.0,8.0,1030.0,250.0,1976.0,0.0,98077.0,47.7427,-122.071,1160.0,10565.0,12.0,44.0,1,0,0.0
4,5.126310e+09,20150115,4.0,2.75,2830.0,8126.0,2.0,0.0,0.0,3.0,8.0,2830.0,0.0,2005.0,0.0,98059.0,47.4863,-122.140,2830.0,7916.0,1.0,15.0,0,0,0.0


In [263]:
# dropping any columns that are not features that I want to use in the final model
holdout = holdout.drop(columns=['id','date','sqft_lot','sqft_above', 'sqft_basement', 'yr_built','yr_renovated', 'zipcode', 'lat','long', 'sqft_living15', 'sqft_lot15'])

In [264]:
# checking that they dropped
holdout.head()

,bedrooms,bathrooms,sqft_living,floors,waterfront,view,condition,grade,month_sold,age,basement,renovated,sqft_waterfront
0,4.0,2.50,2270.0,1.0,0.0,0.0,3.0,8.0,8.0,53.0,1,0,0.0
1,4.0,2.50,2270.0,1.0,0.0,0.0,3.0,8.0,2.0,53.0,1,0,0.0
2,3.0,2.50,1470.0,2.0,0.0,0.0,3.0,8.0,11.0,15.0,1,0,0.0
3,3.0,1.75,1280.0,1.0,0.0,0.0,3.0,8.0,12.0,44.0,1,0,0.0
4,4.0,2.75,2830.0,2.0,0.0,0.0,3.0,8.0,1.0,15.0,0,0,0.0


## Step 3: Predict the holdout set

In [266]:
# predicting on the cleaned up holdout dataframe
final_answer = final_model.predict(holdout)

## Step 4: Export predictions

In [267]:
# final_answer.to_csv('housing_preds_sidney_kung.csv')

final_answer = pd.DataFrame(final_answer, columns=['predictions'])

In [268]:
final_answer.head()

,predictions
0,1.043629e+06
1,1.021922e+06
2,5.480304e+05
3,7.895153e+05
4,7.189543e+05


In [269]:
final_answer.to_csv('housing_preds_sidney_kung.csv')